In [2]:
from cvxopt import matrix, solvers


In [6]:
A = matrix([ [-1.0, 0.0, 0.0],[0.0, -1.0, 0.0], [0.0, 0.0, -1.0],[1.0, 0.0, 0.0],[0.0, 1.0, 0.0], [0.0, 0.0, 1.0], [1.0, 1.0, 1.0]])
b = matrix([ 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 5.0 ])
c = matrix([ 1.0, 1.0, -1.0 ])

In [8]:
sol=solvers.lp(c,A.T,b)

     pcost       dcost       gap    pres   dres   k/t
 0:  1.4000e+00 -1.6200e+01  1e+01  0e+00  2e+00  1e+00
 1:  3.1572e-01 -3.2626e+00  2e+00  1e-16  3e-01  2e-01
 2: -1.8542e+00 -2.9102e+00  9e-01  3e-16  1e-01  1e-01
 3: -1.9983e+00 -2.0098e+00  9e-03  2e-16  1e-03  1e-03
 4: -2.0000e+00 -2.0001e+00  9e-05  3e-16  1e-05  1e-05
 5: -2.0000e+00 -2.0000e+00  9e-07  3e-16  1e-07  1e-07
 6: -2.0000e+00 -2.0000e+00  9e-09  1e-16  1e-09  1e-09
Optimal solution found.


In [11]:
print(sol['x'])

[ 1.95e-10]
[ 1.95e-10]
[ 2.00e+00]



In [13]:
print (sol['primal objective'])

-1.99999999829
